**Install dependencies**

In [ ]:
!pip install -q flask pyngrok transformers accelerate bitsandbytes sentencepiece

**Create folders**

In [ ]:
!mkdir -p templates
!mkdir -p static

**Write app.py**

In [ ]:
%%writefile app.py
# =============================================================
# Project 6: AI-Powered Interactive Interviewer & Feedback Generator
# Merged: Strict question-generation pipeline + stable evaluation/UI
# =============================================================

import os
import re
import uuid
import json
from functools import lru_cache
from typing import List

from flask import Flask, render_template, request, redirect, url_for

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

# ------------------------------
# App config
# ------------------------------
app = Flask(__name__)
app.config["SECRET_KEY"] = "project6-secret"
STORAGE = "sessions"
os.makedirs(STORAGE, exist_ok=True)

# ------------------------------
# Model config
# ------------------------------
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

@lru_cache(maxsize=1)
def load_model():
    """
    Load tokenizer + model once, using BitsAndBytesConfig for quantization.
    Returns tokenizer, model, device.
    """
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb,
        device_map="auto"
    )
    device = model.device
    return tokenizer, model, device

# ------------------------------
# Generation helpers (robust)
# ------------------------------
def clean_text(text: str) -> str:
    # remove odd non-printable / corrupted characters
    text = re.sub(r"[^\x09\x0A\x0D\x20-\x7E\u00A0-\u024F]+", " ", text)
    text = re.sub(r"\s{2,}", " ", text).strip()
    return text

def looks_corrupted(text: str) -> bool:
    if not text:
        return True
    if "�" in text:
        return True
    non_ascii = sum(1 for ch in text if ord(ch) > 127)
    if (non_ascii / max(1, len(text))) > 0.12:
        return True
    return False

def truncate_prompt(prompt: str, max_words: int = 3000) -> str:
    words = prompt.split()
    if len(words) <= max_words:
        return prompt
    # keep the last part (most relevant for context)
    return " ".join(words[-max_words:])

def generate_text(prompt: str,
                  max_new_tokens: int = 700,
                  temperature: float = 0.35,
                  top_p: float = 0.85,
                  deterministic_fallback: bool = True) -> str:
    """
    Generate text using the cached model. Retry with deterministic settings if output looks corrupted.
    """
    tokenizer, model, device = load_model()
    prompt = truncate_prompt(prompt, max_words=3000)

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    do_sample = temperature > 0.0

    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    text = clean_text(text)

    if looks_corrupted(text) and deterministic_fallback:
        try:
            with torch.inference_mode():
                out2 = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=0.0,
                    top_p=1.0,
                    do_sample=False,
                    repetition_penalty=1.0,
                    pad_token_id=tokenizer.eos_token_id
                )
            text2 = tokenizer.decode(out2[0], skip_special_tokens=True)
            text2 = clean_text(text2)
            if not looks_corrupted(text2):
                text = text2
        except Exception:
            pass

    return text

# ------------------------------
# Persistence helpers
# ------------------------------
def save_session(sid: str, data: dict):
    path = os.path.join(STORAGE, f"{sid}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_session(sid: str):
    path = os.path.join(STORAGE, f"{sid}.json")
    if not os.path.exists(path):
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# ------------------------------
# Question generation (strict pipeline copied & adapted)
# ------------------------------
def build_initial_question_prompt(interview_type: str, num_questions: int, role: str, exp_level: str) -> str:
    """
    Initial strict prompt that asks for numbered questions and enforces constraints.
    """
    base_rules = f"""Generate {num_questions} {interview_type.lower()} interview questions for a {exp_level} candidate applying for {role}.

Rules:
- Each question must end with a question mark.
- Each question must be under 20 words.
- Number the questions from 1 to {num_questions} using '1.' '2.' etc.
- Avoid duplicates and paraphrases.
Return ONLY the numbered list of questions (no extra commentary)."""

    if interview_type.lower() == "technical":
        extra = """
Experience level guidance:
- Fresher: use hypothetical phrasing ("How would you...", "What steps would you take..."); avoid asking about prior work.
- Mid-Level: include scenario-based troubleshooting and practical challenges.
- Experienced: focus on architecture, design decisions, scalability, reliability, and trade-offs.
Focus on core fundamentals and practical reasoning."""
    elif interview_type.lower() == "hr":
        extra = """
Experience level guidance:
- Fresher: hypothetical only; avoid asking about prior job experience.
- Mid-Level: focus on collaboration, conflict resolution, accountability.
- Experienced: leadership, stakeholder communication, strategic reasoning.
Avoid technical or domain-specific questions."""
    else:  # Functional
        extra = """
Experience level guidance:
- Fresher: hypothetical only; avoid prior experience references.
- Mid-Level: scenario-focused execution and collaboration questions.
- Experienced: strategic, cross-functional decision-making.
Avoid technical, KPI, or tool-specific questions."""
    return base_rules + "\n\n" + extra

def extract_numbered_questions_from_text(text: str) -> List[str]:
    """
    Extract numbered questions using robust patterns. Falls back to lines ending in '?'.
    """
    if not text:
        return []
    found = re.findall(r'\d+[\.\)]\s*(.+?\?)', text)
    out = [q.strip() for q in found if q.strip().endswith('?') and len(q.split()) >= 3]
    if out:
        # dedupe preserving order
        seen = set()
        res = []
        for q in out:
            if q not in seen:
                res.append(q); seen.add(q)
        return res

    # fallback: plain lines ending in '?'
    lines = [ln.strip() for ln in text.splitlines() if ln.strip().endswith('?')]
    res = [l for l in lines if len(l.split()) >= 3]
    # dedupe
    seen = set()
    final = []
    for q in res:
        if q not in seen:
            final.append(q); seen.add(q)
    return final

def generate_strict_questions(interview_type: str, num_questions: int, role: str, exp_level: str) -> List[str]:
    """
    Full strict pipeline:
    1) Initial generation
    2) If missing -> continuation generation
    3) If still missing -> verification pass to force exactly N questions
    """
    # 1) Initial generation
    initial_prompt = build_initial_question_prompt(interview_type, num_questions, role, exp_level)
    raw = generate_text(initial_prompt, max_new_tokens=900, temperature=0.35, top_p=0.85)
    qs = extract_numbered_questions_from_text(raw)

    # 2) Continuation if needed
    if len(qs) < num_questions:
        remaining = num_questions - len(qs)
        start_num = len(qs) + 1
        cont_prompt = f"""
Generate {remaining} additional unique {interview_type.lower()} questions for a {exp_level} candidate applying for {role}.
Number them {start_num} to {num_questions}. Follow the same rules as before. Return ONLY numbered list.
"""
        raw2 = generate_text(cont_prompt, max_new_tokens=600, temperature=0.3, top_p=0.85)
        new_qs = extract_numbered_questions_from_text(raw2)
        for q in new_qs:
            if q not in qs:
                qs.append(q)
        qs = qs[:num_questions]

    # 3) Verification pass: ensure exactly num_questions; ask model to correct list if mismatch
    if len(qs) != num_questions:
        verify_prompt = f"""
Review this list of {len(qs)} {interview_type.lower()} questions and correct them to exactly {num_questions} questions, following the rules:
- Each question under 20 words
- End with a question mark
- Correct numbering 1 to {num_questions}
- No duplicates
Here is the current list:
{chr(10).join([f"{i+1}. {q}" for i, q in enumerate(qs)])}

Return ONLY the corrected numbered list.
"""
        raw3 = generate_text(verify_prompt, max_new_tokens=700, temperature=0.25, top_p=0.85)
        corrected = extract_numbered_questions_from_text(raw3)
        if corrected and len(corrected) >= num_questions:
            qs = corrected[:num_questions]
        else:
            # fallback: pad with placeholders if still insufficient
            while len(qs) < num_questions:
                qs.append("No valid question generated. Please regenerate.")
            qs = qs[:num_questions]

    # final dedupe & trim
    final = []
    seen = set()
    for q in qs:
        if q not in seen:
            final.append(q); seen.add(q)
    return final[:num_questions]

# ------------------------------
# Answer parsing & evaluation builder
# ------------------------------
def parse_combined_answers(text: str, expected_n: int):
    """
    Parse a single textarea containing all answers in numbered format.
    If numbering is not strictly present, split by lines and map sequentially.
    """
    if not text:
        return [""] * expected_n

    text = text.strip()
    # capture numbered blocks: 1. answer ... 2. answer ...
    pattern = re.compile(r'\s*\d+\s*[\.\)]\s*(.+?)(?=(?:\n\s*\d+\s*[\.\)]\s*)|$)', flags=re.DOTALL)
    matches = pattern.findall(text)
    answers = [m.strip() for m in matches]

    if not answers:
        # fallback: split by double-newline blocks or single newlines
        parts = [p.strip() for p in re.split(r'\n{2,}', text) if p.strip()]
        if not parts:
            parts = [p.strip() for p in text.splitlines() if p.strip()]
        answers = parts[:expected_n]

    # ensure length
    if len(answers) < expected_n:
        answers += [""] * (expected_n - len(answers))
    else:
        answers = answers[:expected_n]
    return answers

def build_evaluation_prompt(role: str, exp_level: str, qa_pairs: List[tuple]) -> str:
    transcript = "\n".join([f"Q{i+1}: {q}\nA{i+1}: {a}\n" for i, (q, a) in enumerate(qa_pairs)])
    prompt = f"""
You are a senior hiring manager evaluating a {exp_level.lower()} {role} candidate.

Below is the full transcript of their interview:

### INTERVIEW TRANSCRIPT START ###
{transcript}
### INTERVIEW TRANSCRIPT END ###

Now, as an experienced HR professional, write a complete and insightful evaluation of the candidate’s overall performance.

📋 Final Interview Evaluation Report

1. Overall Technical Competence:
<explanation must start on a NEW line>

2. Problem-Solving and Analytical Skills:
<explanation must start on a NEW line>

3. Communication and Confidence:
<explanation must start on a NEW line>

4. Behavioral and Team Skills:
<explanation must start on a NEW line>

5. Strengths:
<explanation must start on a NEW line>

6. Areas for Improvement:
<explanation must start on a NEW line>

7. Final Recommendation (Hire / Consider / Reject):
<explanation must start on a NEW line>

8. Overall Score (out of 10):
<explanation must start on a NEW line>

Guidelines:
- Summarize the entire interview, not per question.
- Be objective and recruiter-like.
- Keep it under 250 words.
- Start your response immediately after the next line.
### BEGIN EVALUATION BELOW ###
"""
    return prompt

# ------------------------------
# Routes (Flask)
# ------------------------------
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        role = request.form.get("role", "").strip()
        exp_level = request.form.get("exp_level", "Fresher").strip()
        interview_type = request.form.get("interview_type", "Technical").strip()
        try:
            num_questions = int(request.form.get("num_questions", "5"))
        except Exception:
            num_questions = 5
        num_questions = max(1, min(num_questions, 20))

        sid = str(uuid.uuid4())
        data = {
            "role": role,
            "exp_level": exp_level,
            "interview_type": interview_type,
            "num_questions": num_questions,
            "questions": [],
            "answers": [],
            "evaluation": ""
        }
        save_session(sid, data)

        # Generate strict questions (multi-stage)
        questions = generate_strict_questions(interview_type, num_questions, role, exp_level)

        data["questions"] = questions
        save_session(sid, data)

        return redirect(url_for("answer", sid=sid))

    return render_template("index.html")

@app.route("/answer/<sid>", methods=["GET", "POST"])
def answer(sid):
    data = load_session(sid)
    if not data:
        return "Session not found", 404

    questions = data.get("questions", [])
    if request.method == "POST":
        combined_answers = request.form.get("combined_answers", "")
        answers = parse_combined_answers(combined_answers, len(questions))
        data["answers"] = answers
        save_session(sid, data)
        return redirect(url_for("evaluate", sid=sid))

    return render_template("questions.html", session=data, sid=sid)

@app.route("/evaluate/<sid>", methods=["GET"])
def evaluate(sid):
    data = load_session(sid)
    if not data:
        return "Session not found", 404

    questions = data.get("questions", [])
    answers = data.get("answers", [])
    qa_pairs = list(zip(questions, answers))
    prompt = build_evaluation_prompt(data.get("role", ""), data.get("exp_level", ""), qa_pairs)

    raw_eval = generate_text(prompt, max_new_tokens=700, temperature=0.3, top_p=0.9)
    result = raw_eval.split("### BEGIN EVALUATION BELOW ###")[-1].strip()
    if not result.startswith("📋"):
        result = "📋 Final Interview Evaluation Report\n\n" + result

    data["evaluation"] = result
    save_session(sid, data)

    return render_template("result.html", session=data)

# ------------------------------
# Run app
# ------------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

**templates/index.html**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>AI Interview Setup</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <div class="container">
    <h1>🧠 AI Interview — Setup</h1>

    <form method="post">
      <label>Job Role</label>
      <input name="role" placeholder="e.g., ML Engineer" required/>

      <label>Experience Level</label>
      <select name="exp_level">
        <option>Fresher</option>
        <option>Mid-Level</option>
        <option>Experienced</option>
      </select>

      <label>Interview Type</label>
      <select name="interview_type">
        <option>Technical</option>
        <option>HR</option>
        <option>Functional</option>
      </select>

      <label>Number of Questions (1–20)</label>
      <input type="number" name="num_questions" min="1" max="20" value="5" required/>

      <button type="submit">Generate Questions 🚀</button>
    </form>
  </div>
</body>
</html>


**templates/questions.html**

In [ ]:
%%writefile templates/questions.html
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Interview — Answer</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <style>
    pre.questions { white-space: pre-wrap; background:rgba(255,255,255,0.02); padding:12px; border-radius:8px; color:#e2e5ff; }
    textarea.big { min-height: 260px; }
  </style>
</head>
<body>
  <div class="container">
    <h1>🎤 Interview Questions</h1>

    <div class="card">
      <h3>Questions (Please answer all in the box below)</h3>
      <pre class="questions">
{% for q in session.questions %}
{{ loop.index }}. {{ q }}
{% endfor %}
      </pre>

      <form method="post">
        <label>Enter all answers in the single box below using numbered format (1. ... 2. ...)</label>
        <textarea name="combined_answers" class="big" placeholder="{% for i in range(session.num_questions) %}{{ i+1 }}. \n{% endfor %}" required></textarea>
        <button type="submit">Submit & Evaluate</button>
      </form>
    </div>
  </div>
</body>
</html>

**templates/result.html**

In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Interview Evaluation</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <div class="container">
    <h1>📋 Final Interview Evaluation</h1>

    <div class="card">
      <p><strong>Role:</strong> {{ session.role }} — <strong>Experience:</strong> {{ session.exp_level }}</p>
      <hr/>
      <pre style="white-space:pre-wrap;">{{ session.evaluation }}</pre>
    </div>

    <a href="{{ url_for('index') }}">Start New Interview</a>
  </div>
</body>
</html>


**static/style.css**

In [ ]:
%%writefile static/style.css
body {
  font-family: 'Poppins', Arial, sans-serif;
  background: linear-gradient(135deg,#071120,#0f1724);
  color: #e6f0ff;
  margin: 0;
  padding: 24px 12px;
}

.container {
  max-width: 900px;
  margin: 36px auto;
}

h1 { color: #ffe7b6; margin-bottom: 12px; }
.card {
  background: rgba(255,255,255,0.03);
  padding: 18px;
  border-radius: 12px;
  margin-top: 12px;
}

input, select, textarea {
  width: 100%;
  padding: 10px;
  margin: 8px 0;
  border-radius: 8px;
  border: none;
  background: #ffffff;
  color: #111;
}

button {
  padding: 10px 14px;
  border-radius: 8px;
  border: none;
  background: linear-gradient(90deg,#00c6ff,#0072ff);
  color: white;
  font-weight: 600;
  cursor: pointer;
}
pre { font-family: ui-monospace, SFMono-Regular, Menlo, monospace; }


**Start Flask & ngrok**

In [ ]:
# Kill any existing servers (safe restart)
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
# Start Flask in background
!nohup python app.py > run.log 2>&1 &

In [ ]:
# Start ngrok tunnel (replace with your token or remove if already authed)
from pyngrok import ngrok, conf

# Include your NGROK Auth token here
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# Show logs
!sleep 3 && tail -n 40 run.log